# **Cuarto conjunto de tareas a realizar**

## Paquetes necesarios e inicializaciones

La siguiente práctica consta dos partes principales, la primera de ellas basada en YOLO y en detección de matrículas personas y vehículos y la segunda en OCR (Optical Character Recognition).

Si se tiene una tarjeta gráfica de NVIDIA se puede utilizar la GPU haciendo uso de CUDA, para instalar CUDAv11.6 hacer uso del siguiente script.

In [8]:
import cv2
import math
import yaml
from collections import defaultdict
import numpy as np
from ultralytics import YOLO

In [ ]:
# === 1️⃣ ENTRENAMIENTO DEL MODELO ===
print("🚀 Iniciando entrenamiento del modelo...")

# Carga el modelo base (preentrenado)
model = YOLO("yolo11n.pt")

# Entrena con tu dataset y configuración
results = model.train(
    data="data.yaml",
    imgsz=416,
    batch=4,
    device="cpu",
    epochs=40,
    project="runs/train_custom",
    name="exp1",
    exist_ok=True
)

print("✅ Entrenamiento completado.")

🚀 Iniciando entrenamiento del modelo...
Ultralytics 8.3.222  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i7-12700H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

In [ ]:
# === 2️⃣ CARGAR CLASES PERSONALIZADAS ===
with open("data.yaml", "r") as f:
    data = yaml.safe_load(f)
classNames = data["names"]

print(f"📋 Clases personalizadas: {classNames}")

# === 3️⃣ CARGAR EL MODELO ENTRENADO ===
model = YOLO("runs/train_custom/exp1/weights/best.pt")

# === 4️⃣ TRACKING EN VÍDEO ===
print("🎥 Iniciando detección y tracking...")

video_path = "test.mp4"
vid = cv2.VideoCapture(video_path)

if not vid.isOpened():
    print("❌ Error: no se pudo abrir el video.")
    exit()

track_history = defaultdict(lambda: [])

while True:
    ret, frame = vid.read()
    if not ret:
        print("✅ Fin del video.")
        break

    # Detección con seguimiento
    results = model.track(frame, persist=True, stream=True)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Coordenadas del contenedor
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            class_name = classNames[cls]

            # ID de seguimiento
            track_id = ""
            if hasattr(box, "id") and box.id is not None:
                track_id = str(int(box.id[0].tolist()))

            # Color en función de la clase
            escala = int((cls / len(classNames)) * 255 * 3)
            if escala >= 255 * 2:
                R, G, B = 255, 255, escala - 255 * 2
            elif escala >= 255:
                R, G, B = 255, escala - 255, 0
            else:
                R, G, B = escala, 0, 0

            # Dibujar caja y texto
            cv2.rectangle(frame, (x1, y1), (x2, y2), (R, G, B), 3)
            cv2.putText(
                frame,
                f"{track_id} {class_name} {confidence}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (255, 255, 255),
                2
            )

            # Guardar historial de trayectorias
            if hasattr(box, "id") and box.id is not None:
                tid = int(box.id[0])
                x_center = int((x1 + x2) / 2)
                y_center = int((y1 + y2) / 2)
                track = track_history[tid]
                track.append((x_center, y_center))
                if len(track) > 30:
                    track.pop(0)

                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

    # Mostrar frame
    cv2.imshow("YOLO Tracking - Modelo Personalizado", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q") or key == 27:
        print("🛑 Detección interrumpida por el usuario.")
        break

vid.release()
cv2.destroyAllWindows()

print("🏁 Proceso finalizado correctamente.")


Clases detectadas: ['Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110', 'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop']


FileNotFoundError: [Errno 2] No such file or directory: 'runs\\detect\\train\\weights\\best.pt'